In [1]:
import pandas as pd
pd.options.mode.chained_assignment = None

import numpy as np
import torch

from torch.utils.tensorboard import SummaryWriter
from utils.nn_data_classifier import load_data, Classifier
from utils.preprocess import preprocess, RNNDataset

writer = SummaryWriter(log_dir='logs')



Load the dataset

In [2]:
historical_data = load_data()

historical_data['minute'] = historical_data.time.dt.minute
historical_data = historical_data[historical_data.minute == 0]
historical_data.drop('minute', axis=1, inplace=True)

historical_data

,time,Price,Volume_ETH,Price_BTC,Volume_BTC
128,2016-03-14 10:00:00,13.100,0.008360,414.70,1.500000
201,2016-03-14 14:00:00,14.000,248.973956,414.95,9.077592
318,2016-03-14 19:00:00,14.750,8.442300,414.50,1.679111
468,2016-03-15 00:00:00,12.650,651.351595,415.90,0.481963
490,2016-03-15 01:00:00,12.576,4.558064,415.79,0.615654
...,...,...,...,...,...
2903842,2022-11-12 16:00:00,1274.800,9.022555,16934.00,0.065520
2903902,2022-11-12 17:00:00,1269.900,52.030265,16888.00,0.006411
2903962,2022-11-12 18:00:00,1272.700,0.252397,16899.00,0.006868
2904022,2022-11-12 19:00:00,1269.600,2.803365,16877.00,0.130167


Classify data

In [3]:
time_outlook = 2
classi = Classifier(historical_data)
classified_data = classi.classify_data_strict_time(time_outlook=time_outlook)
classified_data

,time,Price,Volume_ETH,Price_BTC,Volume_BTC,Classification
128,2016-03-14 10:00:00,13.100,0.008360,414.70,1.500000,1
201,2016-03-14 14:00:00,14.000,248.973956,414.95,9.077592,0
318,2016-03-14 19:00:00,14.750,8.442300,414.50,1.679111,0
468,2016-03-15 00:00:00,12.650,651.351595,415.90,0.481963,1
490,2016-03-15 01:00:00,12.576,4.558064,415.79,0.615654,1
...,...,...,...,...,...,...
2903722,2022-11-12 14:00:00,1265.100,1.905119,16851.00,0.161496,1
2903782,2022-11-12 15:00:00,1276.600,21.640138,16923.00,0.381346,0
2903842,2022-11-12 16:00:00,1274.800,9.022555,16934.00,0.065520,0
2903902,2022-11-12 17:00:00,1269.900,52.030265,16888.00,0.006411,0


Sort data

In [4]:
data = classified_data.sort_values('time').reset_index(drop=True)
data

,time,Price,Volume_ETH,Price_BTC,Volume_BTC,Classification
0,2016-03-14 10:00:00,13.100,0.008360,414.70,1.500000,1
1,2016-03-14 14:00:00,14.000,248.973956,414.95,9.077592,0
2,2016-03-14 19:00:00,14.750,8.442300,414.50,1.679111,0
3,2016-03-15 00:00:00,12.650,651.351595,415.90,0.481963,1
4,2016-03-15 01:00:00,12.576,4.558064,415.79,0.615654,1
...,...,...,...,...,...,...
50201,2022-11-12 14:00:00,1265.100,1.905119,16851.00,0.161496,1
50202,2022-11-12 15:00:00,1276.600,21.640138,16923.00,0.381346,0
50203,2022-11-12 16:00:00,1274.800,9.022555,16934.00,0.065520,0
50204,2022-11-12 17:00:00,1269.900,52.030265,16888.00,0.006411,0


In [5]:
SEQ_LEN = 24

In [6]:
processor = preprocess(sequence_length=SEQ_LEN)
training, validation, testing  = processor.preprocess(dataframe=data, validation_size=0.2)

In [7]:
from torch import nn
class RNN_module(nn.Module):
    def __init__(self, hidden_size, input_size, output_size, num_layers, prob):
        super(RNN_module, self).__init__()
        self._num_layers = num_layers
        self._input_size = input_size
        self._hidden_size = hidden_size
        self._output_size = output_size

        self.lstm1 = nn.LSTM(input_size = self._input_size, hidden_size = self._hidden_size, 
                            num_layers = self._num_layers, batch_first = True)
        self._droupout = nn.Dropout(p = prob)
        self._batch_norm = nn.BatchNorm1d(SEQ_LEN)

        self._lstm2 = nn.LSTM(input_size = self._hidden_size, hidden_size = self._hidden_size, 
                            num_layers = self._num_layers, batch_first = True)

        self.fc = nn.Linear(in_features=self._hidden_size, out_features= self._output_size)
        self._relu = nn.ReLU()

    def __str__(self):
        return f"RNN LSTM Model w/ {self._input_size} features and {self._num_layers} layers and {self._hidden_size} of hidden size"

    def forward(self, input):
        
        lstm1_output, (h_n, c_n) = self.lstm1(input)
        droupout_output = self._droupout(lstm1_output)
        out = self._batch_norm(droupout_output)

        out, (h_n, c_n) = self._lstm2(out)
        out = self._droupout(out)

        pred = self.fc(out[:, -1, :])
        pred = self._relu(pred)

        return pred

In [8]:
from torch.utils.data import DataLoader

learning_rate = 0.01
dim_size = training._features.size(dim=-1)
hidden_size = 100
batch_size = 32
number_of_classes = training[1][1].shape[0]
layers = 1

train_dataloader = DataLoader(training, batch_size = batch_size, shuffle = True)
validation_dataloader = DataLoader(validation, batch_size = batch_size, shuffle = True)
test_dataloader = DataLoader(testing, batch_size = batch_size, shuffle = False)

model = RNN_module(hidden_size = hidden_size, input_size = dim_size,
                     output_size = number_of_classes, num_layers = layers, prob=0.2)

In [9]:
labels = training._labels
classes, _ = torch.sort(torch.unique(labels, dim = 0))
weights = torch.zeros((classes.shape[0],))

total = len(data)

for idx, _class in enumerate(classes):
    freq = len(data[data.Classification == _class.item()])
    weights[idx] += 1 - freq/total

weights

tensor([0.5090, 0.4910])

In [10]:
criterion = nn.CrossEntropyLoss(weight=weights)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [11]:
CHECKPOINT_PATH = './models_parameters/LSTM/checkpoints_2/'
BEST_PATH = './models_parameters/LSTM/best_model.pth'

def epoch_training(model, train_dataloader, epoch, total_epochs, optimizer):
    running_correct = 0
    running_loss = 0.0
    n_of_steps = len(train_dataloader)

    for current_batch, (sequence, label) in enumerate(train_dataloader):
        #forward: we are calculating the loss given the parameters
        outputs = model(sequence)
        loss = criterion(input=outputs, target = label.float())

        #backward: lets update the parameters given the current loss
        optimizer.zero_grad() #nullifies the current gradients. If you don't do this, gradients will be added up (you don't want that)
        loss.backward() #computates the bwrd-prop gradient for each model parameter
        optimizer.step() #updates the model current parameter using the gradients.

        predictions = torch.argmax(outputs, 1)
        correct = torch.argmax(label, 1)

        running_correct += (predictions == correct).sum().item()
        running_loss += loss.item()

        if (current_batch + 1) % 50 == 0:
            print(f"epoch {epoch+1}/{total_epochs}, current step(batch): {current_batch+1}/{n_of_steps}, loss = {loss.item():.4f} ")
            writer.add_scalar('training loss: ', running_loss/50, epoch * n_of_steps + current_batch)
            writer.add_scalar('accuracy: ', running_correct/50, epoch * n_of_steps + current_batch)
            running_loss = 0.0
            running_correct = 0

    writer.add_scalar('Epoch loss: ', loss, epoch + 1)


def epoch_validate(model, validation_dataloader, epoch, total_epochs):
    with torch.no_grad():
        n_corrects = 0
        n_samples = 0

        for current_batch, (sequence, label) in enumerate(validation_dataloader):

            #forward: we are calculating the loss given the parameters
            outputs = model(sequence)
            predictions = torch.argmax(outputs, 1)

            n_samples += outputs.shape[0]
            n_corrects += (predictions == label).sum().item()

        acc = 100.0 * n_corrects / n_samples

        print(f"epoch {epoch+1}/{total_epochs} accuracy: {acc}")
        writer.add_scalar('Validation Accuracy: ', acc, epoch+1)

    return acc


def train_loop(model: RNN_module, train_dataloader: DataLoader, validation_dataloader: DataLoader, epochs: int, optimizer: torch.optim):
    
    max_accuracy = 0
    is_best = False

    for epoch in range(epochs):
        epoch_training(model, train_dataloader, epoch, epochs, optimizer)

        accuracy = epoch_validate(model, validation_dataloader, epoch, epochs)

        if accuracy > max_accuracy:
            is_best = True
            max_accuracy = accuracy
        else:
            is_best = False
        
        checkpoint = {
            'epoch': epoch+1,
            'model_state': model.state_dict(),
            'optim_state': optimizer.state_dict()
        }

        if is_best:
            torch.save(checkpoint, BEST_PATH)
        
        torch.save(checkpoint, CHECKPOINT_PATH+f'model_{epoch+1}.pth')

def overfit_batch(model: RNN_module, train_dataloader: DataLoader, epochs: int, optimizer: torch.optim):

    running_correct = 0
    running_loss = 0.0

    sequence, label = next(iter(train_dataloader))
    sequence2, label2 = next(iter(train_dataloader))

    sequence = torch.cat([sequence, sequence2])
    label = torch.cat([label, label2])

    batch_size = len(label)

    for epoch in range(epochs):

        #forward: we are calculating the loss given the parameters
        outputs = model(sequence)
        loss = criterion(input=outputs, target = label.float())

        #backward: lets update the parameters given the current loss
        optimizer.zero_grad() #nullifies the current gradients. If you don't do this, gradients will be added up (you don't want that)
        loss.backward() #computates the bwrd-prop gradient for each model parameter
        optimizer.step() #updates the model current parameter using the gradients.

        predictions = torch.argmax(outputs, 1)
        correct = torch.argmax(label, 1)

        running_correct += (predictions == correct).sum().item() / batch_size
        running_loss += loss.item()

        if (epoch + 1) % 1000 == 0:
            print(f"epoch {epoch+1}/{epochs}, loss = {running_loss/1000:.4f}, accuracy = {running_correct/1000*100:.4f}")

            running_loss = 0.0
            running_correct = 0


In [12]:
c = model.state_dict()

In [13]:
def test_loop(test_dataloader: DataLoader, model: nn.Module):
    with torch.no_grad():
        n_corrects = 0
        n_samples = 0

        for current_batch, (sequence, label) in enumerate(test_dataloader):
            #forward: we are calculating the loss given the parameters
            outputs = model(sequence)
            predictions = torch.argmax(outputs, 1)

            n_samples += outputs.shape[0]
            n_corrects += (predictions == label).sum().item()

            if (current_batch + 1) % 200 == 0:
                print(f"test batch: {current_batch+1}/{len(test_dataloader)}, current accuracy: {100 * n_corrects / n_samples}")

        acc = 100.0 * n_corrects / n_samples
        print(f"final test accuracy: {acc}")


In [14]:
epochs = 10
train_loop(model, train_dataloader=train_dataloader, validation_dataloader = validation_dataloader, epochs=epochs, optimizer=optimizer)
test_loop(test_dataloader=test_dataloader, model=model)
# overfit_batch(model, train_dataloader=train_dataloader, epochs=epochs, optimizer=optimizer)

epoch 1/10, current step(batch): 50/1098, loss = 0.3477 
epoch 1/10, current step(batch): 100/1098, loss = 0.3481 
epoch 1/10, current step(batch): 150/1098, loss = 0.3454 
epoch 1/10, current step(batch): 200/1098, loss = 0.3470 
epoch 1/10, current step(batch): 250/1098, loss = 0.3477 
epoch 1/10, current step(batch): 300/1098, loss = 0.3462 
epoch 1/10, current step(batch): 350/1098, loss = 0.3438 
epoch 1/10, current step(batch): 400/1098, loss = 0.3450 
epoch 1/10, current step(batch): 450/1098, loss = 0.3450 
epoch 1/10, current step(batch): 500/1098, loss = 0.3466 
epoch 1/10, current step(batch): 550/1098, loss = 0.3470 
epoch 1/10, current step(batch): 600/1098, loss = 0.3474 
epoch 1/10, current step(batch): 650/1098, loss = 0.3458 
epoch 1/10, current step(batch): 700/1098, loss = 0.3481 
epoch 1/10, current step(batch): 750/1098, loss = 0.3470 
epoch 1/10, current step(batch): 800/1098, loss = 0.3474 
epoch 1/10, current step(batch): 850/1098, loss = 0.3462 
epoch 1/10, cur

KeyboardInterrupt: 

Using best model in validation

In [15]:
for name, parameter in model.named_parameters():
    print(f'{name} gradient: {parameter.grad.norm():.2e}')

lstm1.weight_ih_l0 gradient: 0.00e+00
lstm1.weight_hh_l0 gradient: 0.00e+00
lstm1.bias_ih_l0 gradient: 0.00e+00
lstm1.bias_hh_l0 gradient: 0.00e+00
_batch_norm.weight gradient: 0.00e+00
_batch_norm.bias gradient: 0.00e+00
_lstm2.weight_ih_l0 gradient: 0.00e+00
_lstm2.weight_hh_l0 gradient: 0.00e+00
_lstm2.bias_ih_l0 gradient: 0.00e+00
_lstm2.bias_hh_l0 gradient: 0.00e+00
fc.weight gradient: 0.00e+00
fc.bias gradient: 0.00e+00


In [ ]:
best_model = RNN_module(hidden_size = hidden_size, input_size = dim_size,
                     output_size = number_of_classes, num_layers = 1)

checkpoint = torch.load(BEST_PATH)
print(f'Model type: {best_model}')
print(f'Best performing model found at {checkpoint["epoch"]}ºepoch')

best_model.load_state_dict(state_dict=checkpoint['model_state'], strict=True)
best_model.eval()

test_loop(test_dataloader=test_dataloader, model=best_model)

In [ ]:
checkpoint = torch.load('./models_parameters/LSTM/checkpoints_2/model_1.pth')

a = checkpoint['model_state']

checkpoint = torch.load('./models_parameters/LSTM/checkpoints_2/model_10.pth')

b = checkpoint['model_state']

In [ ]:
writer.close()